In [97]:
import pandas as pd
import ssl
import numpy as np
ssl._create_default_https_context = ssl._create_unverified_context
df = pd.read_csv("https://raw.githubusercontent.com/cwhitz/git_intro_to_ml/master/heart.csv")

train = df.head(250)
test = df.tail(13)

In [495]:
df.target.value_counts()

1    165
0    138
Name: target, dtype: int64

In [273]:

class Gini: 
    def gini_impurity(self,node):
        return 1-sum([(np.count_nonzero(node==k)/len(node))**2 for k in self.types])
    def gini_impurity_score(self,node_left,node_right):
        m = len(node_left)+len(node_right)
        i = len(node_left)
        return i/m*self.gini_impurity(node_left.target) + (m-i)/m*self.gini_impurity(node_right.target)
class Node:
    def __init__(self,minGini):
        self.col = minGini["col"]
        self.dtype = minGini["dtype"]
        self.split = minGini["split"]
        self.gini = minGini["gini"]
    def getCol(self):
        return self.col
    def getSplit(self):
        return self.split

In [484]:
class DecisionTree(Gini):
    def __init__(self,X,y):
        self.node = pd.concat([X,y],axis=1)
        self.types = np.unique(y)
        self.maxDepth = 10
        self.Tree = self.DecisionTree(df,0)
    def findMajority(self,node):
        counts = [np.count_nonzero(node==k) for k in self.types]
        return counts.index(max(counts))
    def DecisionTree(self,node, depth):
        cols = {col : self.findDtype(self.getColValues(node,col)) for col in df.columns if col != "target" and self.findDtype(self.getColValues(node,col)) != None}
        if depth == self.maxDepth or self.gini_impurity(node)==0 or len(cols)==0:
            return [self.findMajority(node)]
        minGini = {"gini":.5,"col":None,"dtype": None, "split": None}
        for col,dtype in cols.items():
            newGini = self.findGiniImpurityFromCol(node, col,dtype)
            if newGini["gini"] <= minGini["gini"]:
                minGini = newGini
        node_left, node_right = self.getNodes(node,minGini["col"], minGini["dtype"],minGini["split"])
        return [Node(minGini),self.DecisionTree(node_left,depth+1),self.DecisionTree(node_right,depth+1)]
    
    def getColValues(self,node, col):
        return list(node[col].unique())
   
    def getNodesIdx(self,node, col, dtype,split):
        if dtype == "binary":
            return node[col]==split
        if dtype == "numerical":
            return node[col]<=split
        if dtype == "categorical":
            return node[col].isin(split[0])
        
    def findDtype(self, vals):
        if len(vals)==2:
            return "binary"
        elif len(vals)>10:
            return "numerical" 
        elif len(vals)<=10 and len(vals)>2:
            return "categorical"
        return None  
       
    def findGiniImpurityFromCol(self,node, col,dtype):
        vals = self.getColValues(node,col)
        if dtype == "binary":
            splits = vals[:1]
        elif dtype == "numerical":
            splits = sorted(vals)[:-1]
        elif dtype == "categorical":
            splits = self.findCombos(vals)

        nodes = [self.getNodesIdx(node, col, dtype,split) for split in splits]       
        impurities = [self.gini_impurity_score(y[indx],y[~indx]) for indx in  nodes]
        return {"gini":min(impurities),"col":col,"dtype": dtype, "split": splits[impurities.index(min(impurities))]}
  
    
    def findOptions(self,values):
        if len(values) == 0:
            return [values]
        smaller = findOptions(values[1:])
        return smaller + [[values[0]] + x for x in smaller]

    def findCombos(self,vals):
        options = []
        duplicate_options = [([x for x in vals if x not in option], option) for option in self.findOptions(vals)]
        for option in duplicate_options:
            if (option[1],option[0]) not in options and len(option[0]) != len(vals) and len(option[1]) != len(vals):
                options.append(option)
        return options

In [485]:
class Predict:
    def newLevel(self,x, col, dtype,split):
        if dtype == "binary":
            return x[col]==split
        if dtype == "numerical":
            return x[col]<=split
        if dtype == "categorical":
            return x[col].isin(split[1]) 
    def __init__(X,Tree):
        
        while len(Tree) > 1:
            treeLevel = Tree[0]
            direction = 0 if self.newLevel(x,treeLevel.col,treeLevel.dtype,treeLevel.split) else 1
            Tree = Tree[1][direction]

In [486]:
y = df.target
X = df.drop("target",axis=1)
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


In [487]:
X.iloc

In [488]:
Tree = DecisionTree(X,y).Tree

In [489]:
Tree

        [0],
        [<__main__.Node at 0x7fec4cb0f0a0>, [0], [1]]],
       [0]],
      [0]],
     [<__main__.Node at 0x7fec4c921610>, [0], [1]]],
     [<__main__.Node at 0x7fec4c8c33d0>, [1], [0]],
        [0],
        [<__main__.Node at 0x7fec4cbba190>, [0], [0]]],
        [1],
         [<__main__.Node at 0x7fec4c8de6a0>, [1], [0]],
         [<__main__.Node at 0x7fec4c8de9d0>, [0], [0]]]]],
      [1]]]],
     [1],
       [<__main__.Node at 0x7fec4c9ca130>, [1], [1]],
       [1]],
         [<__main__.Node at 0x7fec4c575820>, [0], [0]],
         [0]],
        [1]],
       [<__main__.Node at 0x7fec4c6cc400>, [0], [0]]]]],
      [<__main__.Node at 0x7fec4cb441c0>, [1], [1]],
      [1]],
     [<__main__.Node at 0x7fec4cb44400>, [0], [0]]]]],
      [1],
        [0],
         [<__main__.Node at 0x7fec4d9440a0>, [0], [0]],
         [0]]],
       [1]]],
      [0],
       [<__main__.Node at 0x7fec4c7d4df0>, [0], [0]],
        [<__main__.Node at 0x7fec4cefed00>, [0], [0]],
        [0]]]]],
    

In [471]:
len({1:2,1:2})

1

In [419]:
pd.DataFrame()

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [494]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(max_depth = 1)
classifier.fit(X, y)
print(classifier.tree_.max_depth)
print(classifier)

1
DecisionTreeClassifier(max_depth=1)


In [179]:
pd.read_csv("https://raw.githubusercontent.com/cwhitz/git_intro_to_ml/master/heart.csv")

{1: 1, 2: 2}

In [91]:
hey = 0 if True else 3

In [203]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [393]:
[None,None]

[None, None]

In [195]:
df.to_csv("heart_disease.csv",index=False)

In [ ]:
if len(vals)>10:
                find_gini = [.5,vals[0]]
                for val in vals[1:]:
                    node_left = df[df[col]<val].target
                    node_right = df[df[col]>=val].target
                    impurity = self.gini_impurity_score(node_left,node_right)
                    if impurity < find_gini[0]: find_gini=[impurity,val]